# Psychotherapy Analysis _ Weigthed
This notebook is a slight deviation of the original psychotherapy analysis, as we also weighted the results by the size of the therapy facilities. We wanted to check, whether this results in a different outcome.

In [3]:
# we first load in some libraries
import pandas as pd
import altair as alt
import numpy as np
import scipy.stats

In [4]:
#we read in the data that we got ourselves
data=pd.read_csv("../RawData/Umfrage_Gewichte.csv")
#and zfill for the federal state code
data["Land"]=data["Land"].astype(str).str.zfill(2)
data

,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,Wartezeit auf Sprechstunde MAX,Wartezeit auf Therapie,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen
0,01,Schleswig-Holstein,"Norderstedt, Stadt",22846,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,Schleswig-Holstein,"Kiel, Landeshauptstadt",24103,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,Schleswig-Holstein,"Kiel, Landeshauptstadt",24103,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,Schleswig-Holstein,Wentorf bei Hamburg,21465,1,dicht besiedelt,2,Dr. med. Arnd Schweitzer,0.0,NaN,NaN,1.0,NaN,NaN,NaN,ja,25.0,ja,"vor Kurzem erst eröffnet – Erwartung, dass War..."
4,01,Schleswig-Holstein,"Lübeck, Hansestadt",23539,1,dicht besiedelt,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,16,Thüringen,Gerstungen,99834,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3536,16,Thüringen,Hörselberg-Hainich,99820,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3537,16,Thüringen,Nauendorf,99448,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3538,16,Thüringen,Oberbösa,99718,16,gering besiedelt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are still the same two metrics that matter to us. 
* **The waiting time for a first consultation (Sprechstunde).** This is a first talk between a therapist and a potential patient to figure out whether there is a need for therapy and whether the therapist is the right one to take care of the potential patient.
* **The waiting time for a therapy (Therapy).** This is the waiting time that passes in between the first consultation and the actual start of a therapy of a patient. 

Theoretically, therapists should offer regularly first consultations within 2 weeks or less in order to enable quick check ups. However, in reality a lot of therapists said that they do not see the benefit of a first consultation, if it means that after unpacking their emotional baggage, patients have to wait for months to actually start a therapy. So some therapists decide to only offer a first consultation, if they have an available time slot for said patient. So, the waiting time for a therapy could be unexpectedly low while the waiting time for a first consultation seems to be unreasonably high. To have a more fair metric, we also add the two to create the overall waiting time until the therapy starts.

In [5]:
#then we only take in as a subset all rows where we have an actual entry for the name of a therapist
subset_name=data.dropna(subset=["Praxisname"])
#we turn all data for the waiting time into numerics
subset_name[["Wartezeit auf Sprechstunde","Wartezeit auf Sprechstunde MIN","Wartezeit auf Sprechstunde MAX"]]=subset_name[["Wartezeit auf Sprechstunde","Wartezeit auf Sprechstunde MIN","Wartezeit auf Sprechstunde MAX"]].apply(pd.to_numeric, errors='coerce')
subset_name[["Wartezeit auf Therapie","Wartezeit auf Therapie MIN","Wartezeit auf Therapie MAX"]]=subset_name[["Wartezeit auf Therapie","Wartezeit auf Therapie MIN","Wartezeit auf Therapie MAX"]].apply(pd.to_numeric, errors='coerce')

#and then yield the min for each waiting time for each row
subset_name["Min_Sprechstunde"]=subset_name[["Wartezeit auf Sprechstunde","Wartezeit auf Sprechstunde MIN","Wartezeit auf Sprechstunde MAX"]].min(axis=1)
subset_name["Min_Therapie"]=subset_name[["Wartezeit auf Therapie","Wartezeit auf Therapie MIN","Wartezeit auf Therapie MAX"]].min(axis=1)
#further more we create an overall waiting time "Min_Gesamtwartezeit" by adding the two
subset_name["Min_Gesamtwartezeit"]=subset_name["Min_Sprechstunde"]+subset_name["Min_Therapie"]
subset_name

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,...,Wartezeit auf Therapie MIN,Wartezeit auf Therapie MAX,Führt gar keine Warteliste,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit
3,01,Schleswig-Holstein,Wentorf bei Hamburg,21465,1,dicht besiedelt,2,Dr. med. Arnd Schweitzer,0.0,NaN,...,NaN,NaN,NaN,ja,25.0,ja,"vor Kurzem erst eröffnet – Erwartung, dass War...",0.0,1.0,1.0
28,01,Schleswig-Holstein,"Lübeck, Hansestadt",23539,1,dicht besiedelt,2,Dipl.-Psych. Marian Schmitt,2.0,NaN,...,NaN,NaN,ja,NaN,30.0,ja,"Führt keine Warteliste, weil die Anfrage so gr...",2.0,4.0,6.0
55,01,Schleswig-Holstein,"Tornesch, Stadt",25436,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Nadine Rohwer,NaN,NaN,...,NaN,NaN,NaN,NaN,26.0,ja,"gesetztlich verpflichtet, Sprechstunde innerha...",4.0,72.0,76.0
58,01,Schleswig-Holstein,Altenholz,24161,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Katrin Brinkmann,1.0,NaN,...,NaN,NaN,NaN,NaN,60.0,ja,Wartezeit Therapieplatz 1 Jahr,1.0,48.0,49.0
82,01,Schleswig-Holstein,Brodersby,24398,1,gering besiedelt,2,Dr. med. Sabine Keppel,24.0,NaN,...,NaN,NaN,NaN,NaN,40.0,ja,Vermittlung über Kassenärztliche Vereinigung k...,24.0,24.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,15,Sachsen-Anhalt,Kabelsketal,6184,15,gering besiedelt,1,Praxis Dipl.-Psych.Eileen-Kristina Fokin • Aho...,3.0,NaN,...,NaN,NaN,NaN,NaN,50.0,Ja,NaN,3.0,24.0,27.0
3472,16,Thüringen,"Jena, Stadt",7743,16,dicht besiedelt,1,Frau Dipl.-Psych. Gudrun Biersack (Einzelpraxi...,1.0,NaN,...,8.0,24.0,NaN,NaN,64.0,Ja,NaN,1.0,8.0,9.0
3515,16,Thüringen,Unterwellenborn,7333,16,mittlere Besiedlungsdichte,2,Herrn Dipl.-Psych. Thomas Ullrich (Einzelpraxi...,12.0,3.0,...,24.0,36.0,NaN,0,80.0,Ja,NaN,3.0,24.0,27.0
3516,16,Thüringen,"Zella-Mehlis, Stadt",98544,16,mittlere Besiedlungsdichte,2,Herrn Dipl.-Psych. Andreas Dräbing (Einzelprax...,12.0,3.0,...,3.0,NaN,NaN,NaN,90.0,Ja,Schneller über Terminvergabe,3.0,3.0,6.0


In [6]:
#additionally we check also whether our metrics for therapy slots is only made up of floats
subset_name["Therapieplätze gesamt"].unique()

array([ 25.,  30.,  26.,  60.,  40.,  15.,  28.,  nan,  50.,  90., 120.,
       150.,  20.,  48., 140.,  35.,  70.,  80.,  45.,  75.,  65.,  32.,
       180.,  52., 100.,  63.,  85.,  19., 183.,  34., 137., 110.,  12.,
        18.,  42.,  79.,  78., 220.,  22.,  10.,  62.,  51.,  55.,  36.,
       103.,   6.,  64.])

In [7]:
#now we mulitply all of our metrics in each row with the number of slots
subset_name["Min_Sprechstunde_Gewichtet"]=subset_name["Min_Sprechstunde"]*subset_name["Therapieplätze gesamt"]
subset_name["Min_Therapie_Gewichtet"]=subset_name["Min_Therapie"]*subset_name["Therapieplätze gesamt"]
subset_name["Min_Gesamtwartezeit_Gewichtet"]=subset_name["Min_Gesamtwartezeit"]*subset_name["Therapieplätze gesamt"]
subset_name

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,Land,Bezeichnung,Gemeinde,PLZ,BL,Verstädterung,Sample,Praxisname,Wartezeit auf Sprechstunde,Wartezeit auf Sprechstunde MIN,...,Freie Kapazität,Therapieplätze gesamt,Rückfragen möglich,Anmerkungen,Min_Sprechstunde,Min_Therapie,Min_Gesamtwartezeit,Min_Sprechstunde_Gewichtet,Min_Therapie_Gewichtet,Min_Gesamtwartezeit_Gewichtet
3,01,Schleswig-Holstein,Wentorf bei Hamburg,21465,1,dicht besiedelt,2,Dr. med. Arnd Schweitzer,0.0,NaN,...,ja,25.0,ja,"vor Kurzem erst eröffnet – Erwartung, dass War...",0.0,1.0,1.0,0.0,25.0,25.0
28,01,Schleswig-Holstein,"Lübeck, Hansestadt",23539,1,dicht besiedelt,2,Dipl.-Psych. Marian Schmitt,2.0,NaN,...,NaN,30.0,ja,"Führt keine Warteliste, weil die Anfrage so gr...",2.0,4.0,6.0,60.0,120.0,180.0
55,01,Schleswig-Holstein,"Tornesch, Stadt",25436,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Nadine Rohwer,NaN,NaN,...,NaN,26.0,ja,"gesetztlich verpflichtet, Sprechstunde innerha...",4.0,72.0,76.0,104.0,1872.0,1976.0
58,01,Schleswig-Holstein,Altenholz,24161,1,mittlere Besiedlungsdichte,2,Dipl.-Psych. Katrin Brinkmann,1.0,NaN,...,NaN,60.0,ja,Wartezeit Therapieplatz 1 Jahr,1.0,48.0,49.0,60.0,2880.0,2940.0
82,01,Schleswig-Holstein,Brodersby,24398,1,gering besiedelt,2,Dr. med. Sabine Keppel,24.0,NaN,...,NaN,40.0,ja,Vermittlung über Kassenärztliche Vereinigung k...,24.0,24.0,48.0,960.0,960.0,1920.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,15,Sachsen-Anhalt,Kabelsketal,6184,15,gering besiedelt,1,Praxis Dipl.-Psych.Eileen-Kristina Fokin • Aho...,3.0,NaN,...,NaN,50.0,Ja,NaN,3.0,24.0,27.0,150.0,1200.0,1350.0
3472,16,Thüringen,"Jena, Stadt",7743,16,dicht besiedelt,1,Frau Dipl.-Psych. Gudrun Biersack (Einzelpraxi...,1.0,NaN,...,NaN,64.0,Ja,NaN,1.0,8.0,9.0,64.0,512.0,576.0
3515,16,Thüringen,Unterwellenborn,7333,16,mittlere Besiedlungsdichte,2,Herrn Dipl.-Psych. Thomas Ullrich (Einzelpraxi...,12.0,3.0,...,0,80.0,Ja,NaN,3.0,24.0,27.0,240.0,1920.0,2160.0
3516,16,Thüringen,"Zella-Mehlis, Stadt",98544,16,mittlere Besiedlungsdichte,2,Herrn Dipl.-Psych. Andreas Dräbing (Einzelprax...,12.0,3.0,...,NaN,90.0,Ja,Schneller über Terminvergabe,3.0,3.0,6.0,270.0,270.0,540.0


In [8]:
#then we go through the entire combinations of federal states and types of urbanization
Bundeslaender=subset_name["Land"].unique()
besiedelung=subset_name["Verstädterung"].unique()
#and create a dataframe that holds for each combination the weighted mean
df=pd.DataFrame()
for land in Bundeslaender[:]:
    land_filtered=subset_name[subset_name["Land"]==land]
    for typ in besiedelung:
        land_typ_filtered=land_filtered[land_filtered["Verstädterung"]==typ]
        #these are the weighted means
        sprechstunde_gewichtet=land_typ_filtered["Min_Sprechstunde_Gewichtet"].sum()/land_typ_filtered["Therapieplätze gesamt"].sum()
        therapie_gewichtet=land_typ_filtered["Min_Therapie_Gewichtet"].sum()/land_typ_filtered["Therapieplätze gesamt"].sum()
        gesamtwartezeit_gewichtet=land_typ_filtered["Min_Gesamtwartezeit_Gewichtet"].sum()/land_typ_filtered["Therapieplätze gesamt"].sum()
        #and this is the sum of therapy slots
        Therapieplaetze=land_typ_filtered["Therapieplätze gesamt"].sum()
        #and the number of the sample size
        Samplesize=land_typ_filtered["Sample"].min()
        oneline=pd.Series({"BL":land,"Verstädterung":typ,"Sample":Samplesize,"Therapieplätze gesamt":Therapieplaetze,
         "Min_Sprechstunde_Gewichtet":sprechstunde_gewichtet,"Min_Therapie_Gewichtet":therapie_gewichtet,"Min_Gesamtwartezeit_Gewichtet":gesamtwartezeit_gewichtet})
        df=df.append(oneline,ignore_index=True)
df=df.dropna()
df = df.reset_index(drop=True)

/var/folders/ph/qz6ysgfx5lxf1k82w0hpmj280000gn/T/ipykernel_2816/3486535499.py:11: RuntimeWarning: invalid value encountered in double_scalars
  sprechstunde_gewichtet=land_typ_filtered["Min_Sprechstunde_Gewichtet"].sum()/land_typ_filtered["Therapieplätze gesamt"].sum()
/var/folders/ph/qz6ysgfx5lxf1k82w0hpmj280000gn/T/ipykernel_2816/3486535499.py:12: RuntimeWarning: invalid value encountered in double_scalars
  therapie_gewichtet=land_typ_filtered["Min_Therapie_Gewichtet"].sum()/land_typ_filtered["Therapieplätze gesamt"].sum()
/var/folders/ph/qz6ysgfx5lxf1k82w0hpmj280000gn/T/ipykernel_2816/3486535499.py:13: RuntimeWarning: invalid value encountered in double_scalars
  gesamtwartezeit_gewichtet=land_typ_filtered["Min_Gesamtwartezeit_Gewichtet"].sum()/land_typ_filtered["Therapieplätze gesamt"].sum()


In [9]:
#so this is the resulting table of weighted means per federal state and urbanization type
df

,BL,Verstädterung,Sample,Therapieplätze gesamt,Min_Sprechstunde_Gewichtet,Min_Therapie_Gewichtet,Min_Gesamtwartezeit_Gewichtet
0,01,dicht besiedelt,2.0,55.0,1.090909,2.636364,3.727273
1,01,mittlere Besiedlungsdichte,2.0,86.0,1.906977,55.255814,57.162791
2,01,gering besiedelt,2.0,100.0,38.400000,9.600000,48.000000
3,02,dicht besiedelt,3.0,73.0,1.205479,11.013699,12.219178
4,03,dicht besiedelt,3.0,180.0,1.555556,8.666667,10.222222
5,03,mittlere Besiedlungsdichte,5.0,365.0,6.410959,13.643836,20.054795
6,03,gering besiedelt,3.0,80.0,3.375000,45.000000,48.375000
7,04,dicht besiedelt,1.0,48.0,6.000000,8.000000,14.000000
8,05,dicht besiedelt,12.0,750.0,5.600000,11.746667,17.346667
9,05,mittlere Besiedlungsdichte,9.0,752.0,4.468085,18.930851,23.398936


From our sample, we want to pick the necessary amount of therapists for each federal state and each type of pop density. This is to stratify for size of population in each federal state and to stratify for the amount of people in rural, small town and urban areas. To do this, we go back to a file that we used to estimate the sample number needed for each combination of pop density and federal state. And as our yielded sample is slightly bigger than the needed one, we take a subsample of this dataset for the overall analysis of Germany and for Berlin-Brandenburg

In [10]:
praxenzahl=pd.read_excel("../RawData/AnzahlPraxen.xlsx",sheet_name="Grad_der_Verstädterung",skiprows=6,dtype=str)
praxenzahl_columnlist=["Land","Grad der Verstädterung","Bezeichnung","Fläche in km2",
                       'Bevölkerung insgesamt', 'Bevölkerung männlich', 'Bevölkerung weiblich',"Bevölkerung pro km2",
                       "Fläche_Prozent","Bevölkerung_Prozent","Praxen_gesamt_Sample","Praxen_BB_Sample"]
praxenzahl.columns=praxenzahl_columnlist
praxenzahl_reduced=praxenzahl.dropna(subset=["Grad der Verstädterung","Land"])
praxenzahl_reduced["Praxen_gesamt_Sample"]=pd.to_numeric(praxenzahl_reduced["Praxen_gesamt_Sample"])
praxenzahl_reduced["Praxen_BB_Sample"]=pd.to_numeric(praxenzahl_reduced["Praxen_BB_Sample"])
praxenzahl_reduced["Praxen_gesamt_Sample"]=praxenzahl_reduced["Praxen_gesamt_Sample"].apply(np.ceil)
praxenzahl_reduced["Praxen_BB_Sample"]=praxenzahl_reduced["Praxen_BB_Sample"].apply(np.ceil)
praxenzahl_reduced["Praxen_BB_Sample"]=praxenzahl_reduced["Praxen_BB_Sample"].fillna(0)
praxenzahl_reduced["Sample"]=praxenzahl_reduced["Praxen_BB_Sample"]+praxenzahl_reduced["Praxen_gesamt_Sample"]
praxenzahl_reduced=praxenzahl_reduced.reset_index()
praxenzahl_reduced

/usr/local/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,index,Land,Grad der Verstädterung,Bezeichnung,Fläche in km2,Bevölkerung insgesamt,Bevölkerung männlich,Bevölkerung weiblich,Bevölkerung pro km2,Fläche_Prozent,Bevölkerung_Prozent,Praxen_gesamt_Sample,Praxen_BB_Sample,Sample
0,1,01,01,dicht besiedelt,683.47,835221,405774,429447,1222,4.32457981398745,28.69312491948297,2.0,0.0,2.0
1,2,01,02,mittlere Besiedlungsdichte,2458.45,1236573,601871,634702,503,15.555566804245172,42.48114398591489,2.0,0.0,2.0
2,3,01,03,gering besiedelt,12662.39,839081,418004,421077,66,80.11985338176738,28.825731094602137,2.0,0.0,2.0
3,5,02,01,dicht besiedelt,755.09,1852478,906933,945545,2453,100,100,3.0,0.0,3.0
4,7,03,01,dicht besiedelt,1708.49,1901290,928927,972363,1113,3.581002070223264,23.75596635488749,3.0,0.0,3.0
5,8,03,02,mittlere Besiedlungsdichte,16280.35,3801714,1872376,1929338,234,34.12368059160974,47.50111233683696,5.0,0.0,5.0
6,9,03,03,gering besiedelt,29720.99,2300417,1150153,1150264,77,62.295317338167,28.742921308275548,3.0,0.0,3.0
7,11,04,01,dicht besiedelt,419.62,680130,336390,343740,1621,100,100,1.0,0.0,1.0
8,13,05,01,dicht besiedelt,5933.82,9817621,4798658,5018963,1655,17.39488075468048,54.76880790959506,12.0,0.0,12.0
9,14,05,02,mittlere Besiedlungsdichte,19244.97,6983858,3433386,3550472,363,56.416264443040596,38.960312001236225,9.0,0.0,9.0


In [11]:
#then we stratify the result by taking the result of combination into our sample dataset 
#as many times as the samplesize needs to be
normalized_df=pd.DataFrame()
for index, row in praxenzahl_reduced.iterrows():
    reduced_row=df[(df["BL"]==row["Land"])&(df["Verstädterung"]==row["Bezeichnung"].strip())]
    for i in range(int(row["Praxen_gesamt_Sample"])):
        normalized_df=normalized_df.append(reduced_row,ignore_index=True)

normalized_df

,BL,Verstädterung,Sample,Therapieplätze gesamt,Min_Sprechstunde_Gewichtet,Min_Therapie_Gewichtet,Min_Gesamtwartezeit_Gewichtet
0,01,dicht besiedelt,2.0,55.0,1.090909,2.636364,3.727273
1,01,dicht besiedelt,2.0,55.0,1.090909,2.636364,3.727273
2,01,mittlere Besiedlungsdichte,2.0,86.0,1.906977,55.255814,57.162791
3,01,mittlere Besiedlungsdichte,2.0,86.0,1.906977,55.255814,57.162791
4,01,gering besiedelt,2.0,100.0,38.400000,9.600000,48.000000
...,...,...,...,...,...,...,...
118,15,gering besiedelt,1.0,50.0,3.000000,24.000000,27.000000
119,16,dicht besiedelt,1.0,64.0,1.000000,8.000000,9.000000
120,16,mittlere Besiedlungsdichte,2.0,170.0,3.000000,12.882353,15.882353
121,16,mittlere Besiedlungsdichte,2.0,170.0,3.000000,12.882353,15.882353


In [13]:
#and then we test our hypothesis against this weighted and stratified sample 
from scipy import stats
numberoflongwaiter_Krankenkasse=len(normalized_df[normalized_df["Min_Therapie_Gewichtet"]>=8])
# 3-9 Monate
numberoflongwaiter_BPTK=len(normalized_df[(normalized_df["Min_Therapie_Gewichtet"]>=12) & (normalized_df["Min_Therapie_Gewichtet"]<=36)])

samplesize=len(normalized_df)
claimKrankenkasse=0.16
claimBPTK=0.4
stats.binom_test(numberoflongwaiter_Krankenkasse, n=samplesize, p=claimKrankenkasse, alternative='greater')

1.1259640070836119e-54

In [14]:
stats.binom_test(numberoflongwaiter_BPTK, n=samplesize, p=claimBPTK, alternative='greater')

0.4755539259925201

In [15]:
stadt=normalized_df[normalized_df["Verstädterung"]=="dicht besiedelt"]
land=normalized_df[normalized_df["Verstädterung"]=="gering besiedelt"]

In [16]:
from scipy.stats import mannwhitneyu
mannwhitneyu(stadt["Min_Gesamtwartezeit_Gewichtet"], land["Min_Gesamtwartezeit_Gewichtet"],alternative = 'less')

MannwhitneyuResult(statistic=400.5, pvalue=0.0031268180014720572)

In [17]:
#we do the same stratification process for the different Berlin-Brandenburg-samplesizes
normalized_df_berlinbrandenburg=pd.DataFrame()
for index, row in praxenzahl_reduced.iterrows():
    reduced_row=df[(df["BL"]==row["Land"])&(df["Verstädterung"]==row["Bezeichnung"].strip())]
    for i in range(int(row["Praxen_BB_Sample"])):
        normalized_df_berlinbrandenburg=normalized_df_berlinbrandenburg.append(reduced_row,ignore_index=True)

normalized_df_berlinbrandenburg

,BL,Verstädterung,Sample,Therapieplätze gesamt,Min_Sprechstunde_Gewichtet,Min_Therapie_Gewichtet,Min_Gesamtwartezeit_Gewichtet
0,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
1,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
2,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
3,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
4,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
5,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
6,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
7,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
8,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485
9,11,dicht besiedelt,36.0,1841.0,2.544269,6.883216,9.427485


In [18]:
#and the hypothesis still holds true
berlin=normalized_df_berlinbrandenburg[normalized_df_berlinbrandenburg["BL"]=="11"]
brandenburg=normalized_df_berlinbrandenburg[normalized_df_berlinbrandenburg["BL"]=="12"]
from scipy.stats import mannwhitneyu
mannwhitneyu(berlin["Min_Gesamtwartezeit_Gewichtet"], brandenburg["Min_Gesamtwartezeit_Gewichtet"],alternative = 'less')

MannwhitneyuResult(statistic=217.0, pvalue=0.0032728496982844094)

This is of course a very rough way of including the weights. However, the fact that qualitatively the results do not change much, gives us additional confidence that our results should be valid.

In [20]:
subset_name.to_csv("subset.csv",index=False)